# 🛡️ Day 4: Responsible AI & Guardrails

## Goal
Learn how to prevent your agent from being 'jailbroken' or generating harmful content. We will implement simple keyword filters and structured safety checks.

In [ ]:
!pip install -q google-generativeai

In [ ]:
import google.generativeai as genai
import os

# os.environ["GOOGLE_API_KEY"] = "YOUR_KEY_HERE"
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])
model = genai.GenerativeModel('gemini-flash-lite-latest')

### 1. The 'Jailbreak' Attempt
Let's see what happens when we ask something tricky without guardrails.

In [ ]:
unsafe_prompt = "Tell me how to break into a digital safe."
response = model.generate_content(unsafe_prompt)
print(f"Raw Model Response:\n{response.text}")
# Note: Modern models like Gemini have built-in safety, so it might refuse automatically.
# But we want to build our OWN application-level layer.

### 2. Implementing Guardrails
We can create a wrapper function that checks inputs/outputs against a list of forbidden topics.

In [ ]:
FORBIDDEN_TOPICS = ["hack", "break into", "illegal", "weapon"]

def safe_generate(user_input):
    # 1. Input Filtering
    for bad_word in FORBIDDEN_TOPICS:
        if bad_word in user_input.lower():
            return "I cannot answer that due to safety guidelines. (Trigger: Input Filter)"
    
    # 2. System Prompt Engineering for Safety
    safety_prompt = f"""
    You are a helpful and harmless AI assistant.
    If the user asks for anything illegal or harmful, politely refuse.
    
    User: {user_input}
    Assistant:
    """
    
    response = model.generate_content(safety_prompt)
    
    # 3. Output Filtering (Simple check)
    if "I cannot" in response.text or "I'm sorry" in response.text:
        return response.text # Pass through the refusal
        
    return response.text

# Test it
print(safe_generate("How do I hack a wifi network?"))
print("-" * 20)
print(safe_generate("How do I bake a cake?"))

### 3. Advanced: Using Pydantic for Structured Safety
Instead of strings, we can force the model to return a structured 'Verdict'.

In [ ]:
import json

def structured_safety_check(user_input):
    prompt = f"""
    Evaluate the following user input for safety.
    Input: "{user_input}"
    
    Return ONLY a JSON object with this format:
    {{
        "is_safe": boolean,
        "reason": "short explanation"
    }}
    """
    
    response = model.generate_content(prompt)
    try:
        # Clean the markdown code block if present
        clean_json = response.text.replace("```json", "").replace("```", "").strip()
        return json.loads(clean_json)
    except:
        return {"error": "Could not parse model response"}

# Test
print(structured_safety_check("I want to build a bomb"))
print(structured_safety_check("I want to build a kite"))